Let's install some stuff.

In [ ]:
!pip install opencv-python numpy PyQt5

Let's import some stuff.

In [ ]:
import sys

import cv2
import numpy as np

from PyQt5 import QtCore
from PyQt5 import QtWidgets
from PyQt5 import QtGui
from PyQt5.QtCore import pyqtSignal

Now semantically I need to build this backwords. Let's work on creating the video recording and getting the images out.

In [ ]:
class RecordVideo:
    def __init__(self, camera_port=0):
        self.camera = cv2.VideoCapture(camera_port)
        self.running = False
        
    def run(self):
        self.running = True
        while self.running:
            ret, image = self.camera.read()
            # TODO: detect faces now

Awesome, now let's work on doing some facial detection.

In [ ]:
class FaceDetection:
    def __init__(self, haar_cascade_filepath):
        self.classifier = cv2.CascadeClassifier(haar_cascade_filepath)
        
    def detect_faces(self, image):
        # haarclassifiers work better in black and white
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # TODO: check API
        cv2.equalizeHist(grey_image, grey_image)
        
        faces = self.classifier.detectMultiScale(grey_image,
                                                 scaleFactor=1.1,
                                                 minNeighbors=2,
                                                 flags=cv2.CASCADE_SCALE_IMAGE,
                                                 min_size=min_size)
        
        # TODO: Paint on a surface and add the faces.

Ok so we need something to paint on. This is where we need to switch gears a little bit. We're going to use Qt to paint on. So we'll need to rework some of classes to play nicely with Qt.

We'll start by making our `RecordVideo` a subclass of `QObject`. We'll also create a signal called `image_ready` and have it emit a `np.ndarray` in the `timerEvent`. We'll use it in the `timerEvent` so that we can keep it single threaded.

If that sounds confusing, don't worry. The code isn't that long.

In [ ]:
class RecordVideo(QtCore.QObject):
    image_ready = QtCore.pyqtSignal(np.ndarray)
    
    def __init__(self, camera_port=0, parent=None):
        super().__init__(parent)
        self.camera = cv2.VideoCapture(camera_port)
        self.running = False
        self.timer = QtCore.QBasicTimer()
        
    def start_recording(self):
        self.timer.start(0, self)

    def timerEvent(self, event):
        if (event.timerId() != self.timer.timerId()):
            return

        _, image = self.camera.read()
        self.image_ready.emit(image)

We need to extend our Face Detection as well. We'll make it a `QWidget`, because we want to paint on it. We'll add a new method that converts our ndarray into a `QImage`. We probably can't process each individual frame.

In [ ]:
class FaceDetectionWidget(QtCore.QWidget):
    def __init__(self, haar_cascade_filepath, parent=None):
        super().__init__(parent)
        self.classifier = cv2.CascadeClassifier(haar_cascade_filepath)
        self.timer = QtCore.QBasicTimer()
        self.image = None

    def detect_faces(self, image: np.ndarray):
        # haarclassifiers work better in black and white
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # TODO: check API
        cv2.equalizeHist(grey_image, grey_image)

        faces = self.classifier.detectMultiScale(grey_image,
                                                 scaleFactor=1.1,
                                                 minNeighbors=2,
                                                 flags=cv2.CASCADE_SCALE_IMAGE,
                                                 min_size=min_size)

        return faces

    def image_slot(self, image):
        self.image = image

    def get_qimage(self, image: np.ndarray):
        height, width = image.shape
        bytesPerLine = 3 * width

        image = QImage(image.data,
                       width,
                       height,
                       bytesPerLine,
                       QImage.Format_RGB888)

        image.rgbSwapped()
        return image

    def timerEvent(self, event):
        if (event.timerId() != self.timer.timerId()):
            return

        # TODO: API this better
        self.paint_item(self.image)
        
    def paint_item(self, image: np.ndlayout_widgetarray):
        faces = self.detect_faces(image)
        image = self.get_qimage(image)
        if image.size() != self.size():
            self.setFixedSize(image.size())
        self.update()
        
    def paintEvent(self, event):
        painter = QtGui.QPainter(self)
        painter.drawImage(0,0, self.image)
        self.image = QtGui.QImage()

        # TODO: Paint on a surface and add the faces.

In [ ]:
class MainWidget(QtWidgets.QWidget):
    def __init__(self, haarcascade_filepath, parent=None):
        fp = haarcascade_filepath
        self.face_detection_widget = FaceDetectionWidget(fp)
        
        # TODO: set video port
        self.record_video = RecordVideo()
        
        # TODO: Fix this API
        self.record_video.image_ready.connect(self.face_detection_widget.image_slot)
        
        
        layout = QtWidgets.QVBoxLayout()
        
        layout.addWidget(self.face_detection_widget)
        self.run_button = QtWidgets.QPushButton('Start')
        layout.addWidget(self.run_button)
        
        self.run_button.clicked.connect(self.record_video.start_recording)

In [ ]:
def main():
    app = QtWidgets.QApplication(sys.argv)

    #Button to start the videocapture:

    main_window = QtWidgets.QMainWindow()
    main_window.setCentralWidget(MainWidget())
    main_window.show()
    sys.exit(app.exec_())